# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

# Screenshots

1. Heatmap of all cities from Part I

![Image of Heatmap](https://github.com/Thgiliwt/Python-API-Challenge/blob/main/output_data/Heatmap.png)



2. Marker of hotels for selected cities

![Image of Markers](https://github.com/Thgiliwt/Python-API-Challenge/blob/main/output_data/Marker.PNG)

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df2 = pd.read_csv('../output_data/cityinfo.csv')

df2


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Marzuq,14.4000,46.4667,81.37,26,71,4.07,YE,2021-06-13 17:47:43
1,Kidal,18.4411,1.4078,106.11,9,3,3.96,ML,2021-06-13 17:47:43
2,Sorong,-0.8833,131.2500,78.96,84,90,5.30,ID,2021-06-13 17:47:12
3,Qaanaaq,77.4840,-69.3632,37.22,81,35,1.88,GL,2021-06-13 17:47:43
4,Rikitea,-23.1203,-134.9692,74.95,76,13,10.07,PF,2021-06-13 17:47:44
...,...,...,...,...,...,...,...,...,...
576,Burns Lake,54.2331,-125.7533,60.22,45,57,4.52,CA,2021-06-13 17:50:14
577,‘Ewa Beach,21.3156,-158.0072,75.15,70,90,5.01,US,2021-06-13 17:50:14
578,Radovitskiy,55.1282,39.7956,65.08,92,22,4.52,RU,2021-06-13 17:50:15
579,Līsakovsk,52.5369,62.4936,66.60,35,62,8.66,KZ,2021-06-13 17:47:21


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#access maps with gmap api key
gmaps.configure(api_key = gkey)

location = df2[['Lat','Lng']]

weights = df2['Humidity']

figure_layout = {'width':'100%','height':'400px'}
fig = gmaps.figure(layout = figure_layout,center=(10,20),zoom_level=2)

heat_layer = gmaps.heatmap_layer(location,weights = weights,dissipating=False,max_intensity=100,point_radius=2.5)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='400px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
df3 = df2.loc[(df2['Max Temp']>70)&
              (df2['Max Temp']<80)&
              (df2['Wind Speed']<10)&
              (df2['Cloudiness']==0),:].reset_index(drop=True).dropna()

df3

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mar del Plata,-38.0023,-57.5575,71.10,75,0,5.01,AR,2021-06-13 17:46:35
1,Arrondissement de Troyes,48.3333,4.0833,79.14,38,0,9.22,FR,2021-06-13 17:48:56
2,Jardim,-21.4803,-56.1381,78.51,35,0,5.08,BR,2021-06-13 17:47:08
3,Saint-Pierre,-21.3393,55.4781,73.04,78,0,5.75,RE,2021-06-13 17:45:06
4,Villa Carlos Paz,-31.4241,-64.4978,76.39,48,0,8.99,AR,2021-06-13 17:46:39
5,Gimli,50.6336,-96.9907,78.06,35,0,9.22,CA,2021-06-13 17:50:02
6,San Vicente,-26.6167,-54.1333,71.47,37,0,3.69,AR,2021-06-13 17:47:23


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = df3

df3['Hotel Name']=''

df3

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mar del Plata,-38.0023,-57.5575,71.10,75,0,5.01,AR,2021-06-13 17:46:35,
1,Arrondissement de Troyes,48.3333,4.0833,79.14,38,0,9.22,FR,2021-06-13 17:48:56,
2,Jardim,-21.4803,-56.1381,78.51,35,0,5.08,BR,2021-06-13 17:47:08,
3,Saint-Pierre,-21.3393,55.4781,73.04,78,0,5.75,RE,2021-06-13 17:45:06,
4,Villa Carlos Paz,-31.4241,-64.4978,76.39,48,0,8.99,AR,2021-06-13 17:46:39,
5,Gimli,50.6336,-96.9907,78.06,35,0,9.22,CA,2021-06-13 17:50:02,
6,San Vicente,-26.6167,-54.1333,71.47,37,0,3.69,AR,2021-06-13 17:47:23,


In [6]:
#run an example search of hotel

url_base = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params1 = {'key':gkey,'radius':5000,'keyword':'hotel','location':'-38.0023,-57.5575'}

example1 = requests.get(url_base,params=params1).json()

json.dumps(example1,indent=4,sort_keys=True)

'{\n    "html_attributions": [],\n    "next_page_token": "ATtYBwLBV-_mLvFb4_o86mKDpbz4zXOStJYyg4f2Bu0QJpe41rvIaHpBZbfV41_db7HYlFcBYY430kyPTw9feDNihXy52JMPQN6GMaEhb-utCyyCiCvp1c_1suGiF6NAXVpySjslFyfuhqj_C13Afiz2jCBAwlBI_zW3jcxid8SxB-h0B3LbmxGXu_pYXStBO8Sl2HtDLY33V8xd5p_9olMZ4IP-iztQm0ePhl0dF_Mn4Kj1-mvUUeEV72nrtph9aOcZECYtyZ2ukHxPoQ7hk7w7iSogjYixpd5oF2wHyLoS2-_PI-nW3o4D5sjk6b0oLRy1bE_h616ufmP37j8CNaO8_ZZ6dvy0kQaMBCncyKVYKEvT1mRF3gGfWZAHtJgks-aK2d5HC2J-KO0HO0bnoOifhtCpgVMvMsHkkuBeJcpm4ZlD8Ds9ERU9Kg",\n    "results": [\n        {\n            "business_status": "OPERATIONAL",\n            "geometry": {\n                "location": {\n                    "lat": -38.0199336,\n                    "lng": -57.5263691\n                },\n                "viewport": {\n                    "northeast": {\n                        "lat": -38.01861862010728,\n                        "lng": -57.52505017010728\n                    },\n                    "southwest": {\n                        "lat": 

In [7]:
#API call for hotels located within 5km of each city

url_base = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'key':gkey,'radius':5000,'keyword':'hotel','type':'lodging'}

for index,row in hotel_df.iterrows():
    
    Lat = row['Lat']
    Lng = row['Lng']
    
    params['location'] = f'{Lat},{Lng}'
    
    hotel_detail = requests.get(url_base,params=params).json()
    
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel_detail['results'][0]['name']
        
        print(f"Hotel found for the city of {row['City']}...")
    except:
        print(f"There is no hotel within 5km of {row['City']}, skip to the next...")
    

Hotel found for the city of Mar del Plata...
Hotel found for the city of Arrondissement de Troyes...
Hotel found for the city of Jardim...
Hotel found for the city of Saint-Pierre...
Hotel found for the city of Villa Carlos Paz...
Hotel found for the city of Gimli...
Hotel found for the city of San Vicente...


In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mar del Plata,-38.0023,-57.5575,71.10,75,0,5.01,AR,2021-06-13 17:46:35,Hotel Las Rocas
1,Arrondissement de Troyes,48.3333,4.0833,79.14,38,0,9.22,FR,2021-06-13 17:48:56,Best Western Premier Hôtel De La Poste Troyes ...
2,Jardim,-21.4803,-56.1381,78.51,35,0,5.08,BR,2021-06-13 17:47:08,Hotel Jardim
3,Saint-Pierre,-21.3393,55.4781,73.04,78,0,5.75,RE,2021-06-13 17:45:06,"Villa Delisle, Hotel & Spa"
4,Villa Carlos Paz,-31.4241,-64.4978,76.39,48,0,8.99,AR,2021-06-13 17:46:39,Hotel Pinares del Cerro
5,Gimli,50.6336,-96.9907,78.06,35,0,9.22,CA,2021-06-13 17:50:02,Lakeview Gimli Resort
6,San Vicente,-26.6167,-54.1333,71.47,37,0,3.69,AR,2021-06-13 17:47:23,Hotel Y Restaurante Los Lapachos


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', width='100%'))